In [12]:
import random
from datetime import datetime, timedelta
import tkinter as tk
from tkinter import Button, END, Text


def generate_driver_lists(num_routes):
    drivers_needed = (num_routes + 2) // 5
    return [f"Driver_A{i+1}" for i in range(drivers_needed)], [f"Driver_B{i+1}" for i in range(drivers_needed)]

def genetic_schedule(driver_list_A, driver_list_B, shift_duration_A, shift_duration_B, num_routes, traffic_route_time, max_generations=50, population_size=100):
    def is_time_overlap(start1, end1, start2, end2):
        return max(start1, start2) < min(end1, end2)

    def calculate_route_end(start_time, route_duration):
        return (datetime.combine(datetime.today(), start_time) + route_duration).time()

    def check_driver_schedule_overlap(schedule, driver, start_time, end_time):
        for route_start, route_end in schedule[driver]:
            if is_time_overlap(route_start, route_end, start_time, end_time):
                return True
        return False

    def fitness(schedule):
        penalties = 0
        # Проверка на пересечение времени маршрутов для одного водителя
        for driver, routes in schedule.items():
            for start, end in routes:
                if check_driver_schedule_overlap(schedule, driver, start, end):
                    penalties += 1  # Увеличиваем штраф за пересечение

        return -penalties

    def complete_shift(schedule, driver, shift_duration):
        total_worked_time = sum((datetime.combine(datetime.today(), end) - datetime.combine(datetime.today(), start)).seconds for start, end in schedule[driver])
        required_time = shift_duration.total_seconds() - total_worked_time

        if required_time > 0:
            last_end_time = schedule[driver][-1][1] if schedule[driver] else datetime.strptime("06:00", "%H:%M").time()
            new_end_time = (datetime.combine(datetime.today(), last_end_time) + timedelta(seconds=required_time)).time()
            if new_end_time <= datetime.strptime("20:00", "%H:%M").time():
                schedule[driver].append((last_end_time, new_end_time))

    def create_population():
        population = []
        for _ in range(population_size):
            schedule = {driver: [] for driver in driver_list_A + driver_list_B}
            for driver in schedule.keys():
                while len(schedule[driver]) < 5:  # Ограничиваем количество рейсов для каждого водителя
                    start_time = datetime.strptime(f"{random.randint(6, 20)}:00", "%H:%M").time()
                    end_time = calculate_route_end(start_time, traffic_route_time)
                    if not check_driver_schedule_overlap(schedule, driver, start_time, end_time):  # Проверяем на пересечение
                        schedule[driver].append((start_time, end_time))
            population.append(schedule)
        return population

    def crossover(parent1, parent2):
        child = {}
        for driver in driver_list_A + driver_list_B:
            child[driver] = parent1[driver] if random.random() > 0.5 else parent2[driver]
        return child

    def mutate(schedule):
        driver = random.choice(driver_list_A + driver_list_B)
        if schedule[driver]:
            schedule[driver].pop(random.randint(0, len(schedule[driver]) - 1))
        start_time = datetime.strptime(f"{random.randint(6, 20)}:00", "%H:%M").time()
        end_time = calculate_route_end(start_time, traffic_route_time)
        if not check_driver_schedule_overlap(schedule, driver, start_time, end_time):  # Проверяем на пересечение
            schedule[driver].append((start_time, end_time))
        complete_shift(schedule, driver, shift_duration_A if driver in driver_list_A else shift_duration_B)

    population = create_population()
    for _ in range(max_generations):
        population.sort(key=fitness, reverse=True)
        next_population = population[:10]

        while len(next_population) < population_size:
            parent1, parent2 = random.choices(population[:50], k=2)
            child = crossover(parent1, parent2)
            if random.random() < 0.2:
                mutate(child)
            next_population.append(child)

        population = next_population

    return max(population, key=fitness)

# Отображение расписания
def display_schedule(schedule):
    schedule_text.delete(1.0, END)
    for driver, routes in schedule.items():
        # Сортируем рейсы по времени
        sorted_routes = sorted(routes, key=lambda x: x[0])  # Сортировка по времени начала
        schedule_text.insert(END, f"Водитель: {driver}\n")
        for start, end in sorted_routes:
            schedule_text.insert(END, f"  Рейс с {start.strftime('%H:%M')} до {end.strftime('%H:%M')}\n")
        schedule_text.insert(END, "\n")

# Создание интерфейса
root = tk.Tk()
root.title("Генетическое расписание")
root.geometry("600x400")

button_frame = tk.Frame(root)
button_frame.pack(pady=10)

num_routes_entry = tk.Entry(button_frame, width=10)
num_routes_entry.pack(pady=5)

schedule_text = Text(root, width=70, height=15)
schedule_text.pack(pady=10)

# Генерация расписания для водителей
def generate_genetic_schedule():
    try:
        num_routes = int(num_routes_entry.get())
        drivers_A, drivers_B = generate_driver_lists(num_routes)  # Генерация списков водителей
        if not drivers_A and not drivers_B:
            schedule_text.insert(END, "\nНет маршрутов.\n")
            return
        best_schedule = genetic_schedule(drivers_A, drivers_B, shift_duration_A, shift_duration_B, num_routes, traffic_route_time)
        display_schedule(best_schedule)
    except ValueError:
        schedule_text.insert(END, "\nОшибка: Введите число маршрутов.\n")

genetic_button = Button(button_frame, text="Создать генерасписание", command=generate_genetic_schedule, bg="white", fg="#3D0071", font=("Helvetica", 12), relief="solid", bd=2)
genetic_button.pack(pady=5)

# Списки водителей и их смены
traffic_route_time = timedelta(minutes=90)  # Средняя длительность маршрута
shift_duration_A = timedelta(hours=8)  # Длительность смены для водителей типа A
shift_duration_B = timedelta(hours=12)  # Длительность смены для водителей типа B

root.mainloop()

In [13]:
import random
from datetime import datetime, timedelta
import tkinter as tk
from tkinter import Button, END, Text

# Определение параметров
class Driver:
    def __init__(self, name):
        self.name = name
        self.routes = []

    def add_route(self, start_time, end_time):
        self.routes.append((start_time, end_time))

    def is_available(self, start_time, end_time):
        for route_start, route_end in self.routes:
            if (start_time < route_end and end_time > route_start):  # Проверка на пересечение
                return False
        return True

def generate_drivers(num):
    return [Driver(f"Driver_{i+1}") for i in range(num)]

def create_schedule(drivers, num_routes, traffic_route_time):
    available_times = []
    for _ in range(num_routes):
        start_hour = random.randint(6, 20)  # Генерация случайного времени начала рейса
        start_time = datetime.strptime(f"{start_hour}:00", "%H:%M").time()
        end_time = (datetime.combine(datetime.today(), start_time) + traffic_route_time).time()
        available_times.append((start_time, end_time))

    for start_time, end_time in available_times:
        assigned = False
        for driver in drivers:
            if driver.is_available(start_time, end_time):
                driver.add_route(start_time, end_time)
                assigned = True
                break
        if not assigned:
            print(f"Не удалось назначить рейс с {start_time} до {end_time}")

def display_schedule(drivers):
    schedule_text.delete(1.0, END)
    for driver in drivers:
        # Сортируем рейсы по времени
        sorted_routes = sorted(driver.routes, key=lambda x: x[0])  # Сортировка по времени начала
        schedule_text.insert(END, f"Водитель: {driver.name}\n")
        for start, end in sorted_routes:
            schedule_text.insert(END, f"  Рейс с {start.strftime('%H:%M')} до {end.strftime('%H:%M')}\n")
        schedule_text.insert(END, "\n")

# Создание интерфейса
root = tk.Tk()
root.title("Расписание рейсов")
root.geometry("600x400")

button_frame = tk.Frame(root)
button_frame.pack(pady=10)

num_routes_entry = tk.Entry(button_frame, width=10)
num_routes_entry.pack(pady=5)

schedule_text = Text(root, width=70, height=15)
schedule_text.pack(pady=10)

# Генерация расписания для водителей
def generate_schedule():
    try:
        num_routes = int(num_routes_entry.get())
        num_drivers = (num_routes + 2) // 5  # Количество водителей
        drivers = generate_drivers(num_drivers)
        traffic_route_time = timedelta(minutes=90)  # Средняя длительность маршрута
        create_schedule(drivers, num_routes, traffic_route_time)
        display_schedule(drivers)
    except ValueError:
        schedule_text.insert(END, "\nОшибка: Введите число маршрутов.\n")

# Кнопка для запуска
generate_button = Button(button_frame, text="Создать расписание", command=generate_schedule, bg="white", fg="#3D0071", font=("Helvetica", 12), relief="solid", bd=2)
generate_button.pack(pady=5)

root.mainloop()

Не удалось назначить рейс с 15:00:00 до 16:30:00
Не удалось назначить рейс с 15:00:00 до 16:30:00
